# JS06 - Building RESTQL APIs

Query as API (aka RESTQL) enables developers to quickly convert saved C8QL queries into geo-distributed REST APIs. This eliminates the need for separate backend servers & containers for CRUD operations.

## Step 06-A - Import Libraries to Workbook

In [1]:
const jsc8 = require("jsc8");

## Step 06-B - Define Login Credentials

In [2]:
const guest_mail = "email";
const guest_password = "password";

## Step 06-C - Define Variables and assign Values for Workbook


In [3]:
const fed_url = "verizon.5g.macrometa.io";
const geo_fabric = "_system";
let collection_name = "person";

## Step 06-D - Connect to GDN

Now that you have added your login credentials, lets connect to GDN

In [4]:
console.log("\n ------- CONNECTION SETUP  ------");
console.log(`tenant: ${guest_mail}, geofabric: ${geo_fabric}`);
// Simple Way
// const client = new jsc8({url: fed_url, token: "XXXX", fabricName: geo_fabric});

// ----- OR -----
// const client = new jsc8({url: fed_url, apiKey: "XXXX", fabricName: geo_fabric});

// To use advanced options
const client = new jsc8(fed_url);
client
  .login(guest_mail, guest_password)
  .then((result) => console.log("Login successfully", result))
  .catch((err) => console.error("Error while login", err.message));


## Step 06-E - Create a Geo Replicated Collection

Before we create the RESTQL APIs lets create a collection that we can work with

In [11]:
console.log("\n ------- CREATE GEO-REPLICATED COLLECTION  ------");

async function createCollection() {
  try {
    // Create a new collection if it does not exist
    if (await client.hasCollection(collection_name)) {
      console.log("Collection exists with name ", collection_name);
      return;
    }
    await client.createKVCollection(collection_name);
    console.log("Collection created! ", collection_name);
  } catch (e) {
    return "Collection creation did not succeed due to " + e;
  }
}

createCollection();

## Step 06-F - Create Rest APIs (RESTQLs)

### Step 06-F1 - Defining our Queries

In this section we are going to define five RESTQL Query APIs

In [6]:
let parameter = { firstname: "", lastname: "", email: "", zipcode: "" };
let insert_data = {
  query: {
    name: "insertRecord",
    value: `INSERT {'firstname':@firstname, 'lastname':@lastname, 'email':@email, 'zipcode':@zipcode, '_key': 'abc'} IN ${collection_name}`,
    parameter: parameter,
  },
};
let get_data = {
  query: {
    name: "getRecords",
    value: `FOR doc IN ${collection_name} RETURN doc`,
  },
};

let update_data = {
  query: {
    name: "updateRecord",
    value: `UPDATE 'abc' WITH { \"lastname\": \"cena\" } IN ${collection_name}`,
  },
};

let delete_data = {
  query: {
    name: "deleteRecord",
    value: `REMOVE 'abc' IN ${collection_name}`,
  },
};

let get_count = {
  query: {
    name: "countRecords",
    value: `RETURN COUNT(FOR doc IN ${collection_name} RETURN 1)`,
  },
};


### Step 06-F1 - Creating the RESTQL APIs

The Following Code will create a set of RESTQL APIs in GDN - once complete you can check that they exist from your console access, under Queries, and navigate to "Saved Queries"

In [8]:
async function createRestQL() {
  console.log("\n ------- CREATE RESTQLs  ------");
  await client.createRestql(
    insert_data.query.name.toString(),
    insert_data.query.value.toString(),
    insert_data.query.parameter
  ); // name: insertRecord

  await client.createRestql(
    get_data.query.name.toString(),
    get_data.query.value.toString(),
    {}
  ); // name: getRecords

  await client.createRestql(
    update_data.query.name.toString(),
    update_data.query.value.toString(),
    {}
  ); // name: updateRecord

  await client.createRestql(
    delete_data.query.name.toString(),
    delete_data.query.value.toString(),
    {}
  ); // name: deleteRecord

  await client.createRestql(
    get_count.query.name.toString(),
    get_count.query.value.toString(),
    {}
  ); // name: countRecords
}

createRestQL();


## Step 06-G - Execute RESTQLs

**Note: we are going to skip testing the "deleteRecord" API in this section - so that we can demonstrate how to update a RESTQL Query API first.**

### Step 06-G1 - Test the "insertRecord" RESTQL

In [12]:
async function executeRestQL() {
  console.log("\n ------- EXECUTE RESTQLs ------");
  console.log("\n a. Insert Data");
  let resp = await client.executeRestql(insert_data.query.name.toString(), {
    firstname: "john",
    lastname: "doe",
    email: "john.doe@macrometa.io",
    zipcode: "511037",
  });
  console.log(resp.result);

  console.log("\n b. Get Data");
  resp = await client.executeRestql(get_data.query.name.toString(), {});
  console.log(resp.result);
}
executeRestQL();


### Step 06-G2 - Test the "updateRecord" RESTQL

In [13]:
async function executeRestQL() {
  console.log("\n ------- EXECUTE RESTQLs ------");

  console.log("\n c. Update Data");
  resp = await client.executeRestql(update_data.query.name.toString(), {});
  console.log(resp.result);

  console.log("\n d. Get Data");
  resp = await client.executeRestql(get_data.query.name.toString(), {});
  console.log(resp.result);
}
executeRestQL();

### Step 06-G3 - Test the "getRecords" RESTQL

In [14]:
async function executeRestQL() {
  console.log("\n ------- EXECUTE RESTQLs ------");

  console.log("\n d. Get Records");
  resp = await client.executeRestql(get_data.query.name.toString(), {});
  console.log(resp.result);
}
executeRestQL();

### Step 06-G4 - Test the "countRecords" RESTQL

In [15]:
async function executeRestQL() {
  console.log("\n ------- EXECUTE RESTQLs ------");

  console.log("\n e. Count Records");
  resp = await client.executeRestql(get_count.query.name.toString(), {});
  console.log(resp.result);
}
executeRestQL();

## Step 06-H - Updating a RESTQL API

### Step 06-H1 - Test that we can query GDN and see the document

A common requirement when building an application is to make a change to an API and update it. 

To demonstrate this, lets first query the database using the "execute_query" method we learnt in an earlier section

Once run, we should see the document in the cell output in JSON format so we can read it easily.

In [16]:
console.log('Get docs in "person" collection');

async function c8Queries() {
  let docs = await client.executeQuery(
    `FOR doc IN ${collection_name} RETURN doc`
  );
  console.log(docs);
}

c8Queries();

### Step 06-H2 - Updating a RESTQL API

The follow cell show us how to add a new parameter to the Query, in this example we are going to add a the parameter "phone" to the query.

In [31]:
value = `INSERT {'firstname':@firstname, 'lastname':@lastname, 'email':@email, 'zipcode':@zipcode, 'phone': @phone} IN ${collection_name}`;

insert_data = {
  query: { name: "insertRecord", parameter: parameter, value: value },
};

async function updateRestQL() {
  console.log("\n ------- UPDATE RESTQLs  ------");
  response = await client.updateRestql(
    insert_data.query.name,
    insert_data.query.value,
    insert_data.query.parameter
  );
  console.log(response);
}

updateRestQL();


### Step 06-H3 - Testing the updated RESTQL API

Now that we have updated the Query, lets add a new record using it to test that it works.

In [26]:
async function executeRestQL() {
  console.log("\n ------- EXECUTE RESTQLs ------");
  console.log("\n Insert data with updated query worker");
  let resp = await client.executeRestql(query_name, {
    firstname: "john",
    lastname: "doe",
    email: "john.doe@macrometa.io",
    zipcode: "511037",
    phone: "213-555-9578",
  });
  console.log(resp.result);
}

executeRestQL();


### Step 06-H4 - Checking the data in the collection

Now lets check that the data has been updated in the record by using the "execute_query" method - we should see two documents. 

1. the original document without a phone number, and
2. a second document, but now with a phone number added

**Hint: If you would like to return to "Step 06-G4 - Test the "countRecords" RESTQL" and re run the code, you should see the count value increase by 1**

In [27]:
console.log(
  'Get docs in "person" collection, after executing the updated query'
);

async function c8Queries() {
  let docs = await client.executeQuery(
    `FOR doc IN ${collection_name} RETURN doc`
  );
  console.log(docs);
}

c8Queries();

## Step 06-I - Deleting a record using a RESTQL API

In [28]:
async function executeRestQL() {
  console.log("Delete data....");
  let resp = await client.executeRestql("deleteRecord");
  console.log(resp.result);
}

executeRestQL();

## Step 06-J - Delete RESTQLs

We might want to delete our RESTQL APIs completely.

For each RESTQL we want to remove we use the "deleteRestql" method as shown in the cell below

In [32]:
async function deleteRestQL() {
  console.log("\n ------- DELETE RESTQLs ------");
  await client.deleteRestql(insert_data.query.name.toString());
  await client.deleteRestql(get_data.query.name.toString());
  await client.deleteRestql(update_data.query.name.toString());
  await client.deleteRestql(get_data.query.name.toString());
  await client.deleteRestql(get_count.query.name.toString());
  await client.deleteRestql(delete_data.query.name.toString());
  console.log("\n ------- DONE  ------");
}
deleteRestQL();

## Step 06-K - Delete the Collection

Now let us remove the collection we added during this Tutorial

In [33]:
async function deleteCollection() {
  try {
    await client.deleteCollection(collection_name);
    console.log("Collection Deleted: ", collection_name);
  } catch (err) {
    console.log("Error while deleting collection ", err.message);
  }
}

deleteCollection();

## Section Completed!

Congratulations!, another tutorial completed.

**Note: To use the created RESTQL APIs you should use the API reference documentation for detailed instructions.**

e.g. /_fabric/{fabric}/_api/restql/{name} 

where {name} is the name of your RESTQL API e.g. "insertRecord"

